<a href="https://colab.research.google.com/github/vvivvi/kaggle-c1/blob/master/KaggleCompetition_LB_probing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget -O competitive-data-science-predict-future-sales.zip https://github.com/vvivvi/kaggle-c1/blob/master/competitive-data-science-predict-future-sales.zip?raw=true
!mkdir competitive-data-science-predict-future-sales
!unzip -o competitive-data-science-predict-future-sales.zip -d competitive-data-science-predict-future-sales

--2020-02-05 21:14:25--  https://github.com/vvivvi/kaggle-c1/blob/master/competitive-data-science-predict-future-sales.zip?raw=true
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/vvivvi/kaggle-c1/raw/master/competitive-data-science-predict-future-sales.zip [following]
--2020-02-05 21:14:26--  https://github.com/vvivvi/kaggle-c1/raw/master/competitive-data-science-predict-future-sales.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vvivvi/kaggle-c1/master/competitive-data-science-predict-future-sales.zip [following]
--2020-02-05 21:14:26--  https://raw.githubusercontent.com/vvivvi/kaggle-c1/master/competitive-data-science-predict-future-sales.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.

In [0]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 

Read sample submission into pandas dataframe


In [0]:
DATA_FOLDER = 'competitive-data-science-predict-future-sales'

transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
sample_submission = pd.read_csv(os.path.join(DATA_FOLDER, 'sample_submission.csv'))
test_spec = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))

Let's try to figure out how the test set is split into public and private parts. We have already figured out that the test specification enumerates all query pairs of form (shop, item), where shop is one of the shops active (=with transaction data) and item \in {test_items}. Test items is the set of all items involved in any transaction during Dec 15.

Furthermore, it is known that test data splits about 35:65 between public and private parts.

The division could be to 1) split the queried pairs disjointly into public and private parts, or 2) split the individual transactions into public and private, even within a queried pair. In 2) the predictions and score would have to be scaled appropriately for the evaluation.

Let's probe the leaderboard to find out which is the case. We are allowed 10 submissions/day via course assignment and 5 submissions/day via Kaggle competition.

We may be able to see if the split in deed is either 1) or 2) by submitting otherwise zeros but setting one random id to a large value. In case 1) there should be approximately similar change in score 35% of the time, and no change 65% of time. In 2) there should be a score change practically every time.

Let's generate 13 pairs + baseline to see, what might be the case:


Find constant that minimizes clipped rmse for dec14

In [0]:
def write_predictions_by_array(array, filename):
  df=pd.DataFrame(array)
  df.columns=['item_cnt_month']
  df.to_csv(os.path.join(DATA_FOLDER, filename), index_label='ID')


In [0]:
LARGE_VALUE = 25*sample_submission.shape[0]
pred = np.zeros(sample_submission.shape[0])

write_predictions_by_array(pred, 'baseline0.csv')

np.random.seed(43)

n_random=13

index_to_perturb = np.random.randint(low=0, high = sample_submission.shape[0], size = n_random)

for idx in index_to_perturb:
  pred = np.zeros(sample_submission.shape[0])
  pred[idx] = LARGE_VALUE
  fn = 'single-perturbed-'+str(idx)+'.csv'
  write_predictions_by_array(pred, fn)



| Perturbed index|Assignment score (public/private)| Kaggle public LB score|
|---|---|---|
baseline | 1.250/1.237 | |
105303 | 1.250 / 14311 | 1.250 |
125205 | | 1.250 |
129082 ||1.250|
14148 || 1.250|
147394 || 19659|
150848 | 19659 /1.237 |
28507 | 1.250 / 14312 |
|---|---|---|
137699 | 1.250 / 14312 ||
137700 | 19659 / 1.237 ||
<= 137699 | 

Now it seems clear that the split is really 1). It looks like private part would be first iin its entirety and thereafter the public part.


Lets try to find the splitting point. 0.65*42 shops ~ 27 shops
27*5100=137700

Try perturbing indices 137699 and 137700.

Seems we found the split point. Let's confirm

Confirmation didn't work






In [0]:
test_spec.iloc[137699:,:]

,ID,shop_id,item_id
137699,137699,48,969
137700,137700,57,5037
137701,137701,57,5320
137702,137702,57,5233
137703,137703,57,5232
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [0]:
LARGE_VALUE = 25*sample_submission.shape[0]

index_to_perturb = [137699, 137700]

for idx in index_to_perturb:
  pred = np.zeros(sample_submission.shape[0])
  pred[idx] = LARGE_VALUE
  fn = 'single-perturbed-'+str(idx)+'.csv'
  write_predictions_by_array(pred, fn)


In [0]:
LARGE_VALUE = 25
pred = np.zeros(sample_submission.shape[0])
pred[0:137700] = LARGE_VALUE
fn = 'private-perturbed-'+str(idx)+'.csv'
write_predictions_by_array(pred, fn)

LARGE_VALUE = 25
pred = np.zeros(sample_submission.shape[0])
pred[137700:] = LARGE_VALUE
fn = 'public-perturbed-'+str(idx)+'.csv'
write_predictions_by_array(pred, fn)

